In [ ]:
from grid.robot.arm.isaac_arm import IsaacArm
armisaacsim_0 = IsaacArm()

In [ ]:
# Activate the robot client

armisaacsim_0.run()

In [ ]:
from grid.robot.arm.isaac_arm import IsaacArm
from grid.utils.types import Position, Orientation
from grid.model.perception.detection.owlv2 import OWLv2
from grid.utils.types import Velocity
from grid.utils import log

import time
import numpy as np

# Define the target object for detection
obj = "green cube"

# Load the object detection model
det_model = OWLv2()

# -----------------------------
# Robot and Camera Parameters
# -----------------------------
goal_quat_np = np.array([0.0, 0.0, 0.0, 1.0])  # Desired orientation (quaternion)
step_size = 0.1    # Step size for XY movements
step_size_z = 0.05 # Step size for Z movements

# Camera intrinsic parameters
focal_length = 24.0         # in mm
horizontal_aperture = 20.955  # in mm
image_width = 256.0         # in pixels
image_height = 256.0        # in pixels

# Camera extrinsic parameters: translation and rotation
t = [1.5, 0, 0.7]   # Translation vector
q = [0, -0.3, 0, 1]  # Quaternion for rotation

# Compute effective focal lengths in x and y
fx = (focal_length / horizontal_aperture) * image_width
fy = (focal_length / horizontal_aperture) * image_height

# Compute the optical center of the image
cx = image_width / 2
cy = image_height / 2

# Desired goal height (z position) for the arm's end-effector
goal_z = 0.2

# -----------------------------
# Build the World-to-Camera Transformation Matrix
# -----------------------------
# Compute the rotation matrix from the quaternion 'q'
rot_matrix = np.array([
    [1 - 2*(q[1]**2 + q[2]**2), 2*(q[0]*q[1] - q[2]*q[3]), 2*(q[0]*q[2] + q[1]*q[3])],
    [2*(q[0]*q[1] + q[2]*q[3]), 1 - 2*(q[0]**2 + q[2]**2), 2*(q[1]*q[2] - q[0]*q[3])],
    [2*(q[0]*q[2] - q[1]*q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 1 - 2*(q[0]**2 + q[1]**2)]
])

# Create the homogeneous transformation matrix (world from camera)
T_world_from_cam = np.array([
    [rot_matrix[0, 0], rot_matrix[0, 1], rot_matrix[0, 2], t[0]],
    [rot_matrix[1, 0], rot_matrix[1, 1], rot_matrix[1, 2], t[1]],
    [rot_matrix[2, 0], rot_matrix[2, 1], rot_matrix[2, 2], t[2]],
    [0, 0, 0, 1]
])


In [ ]:
def get_box_center(box, image_size):
    """
    Calculate the center of a bounding box in [x_min, y_min, x_max, y_max] format.

    Args:
        box (list or np.ndarray): Bounding box coordinates.
        image_size (list or tuple): Image dimensions [width, height].

    Returns:
        tuple: (x_center, y_center). Returns (-1, -1) if the box is too large.
    """
    x_min, y_min, x_max, y_max = box
    # Discard boxes that are too large relative to the image area
    if image_size[0] * image_size[1] * 0.2 < (x_max - x_min) * (y_max - y_min):
        return -1, -1
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    return x_center, y_center


# -----------------------------
# Retrieve Camera Images
# -----------------------------
rgb, depth = None, None
while rgb is None or rgb.data is None or depth is None or depth.data is None:
    print("Waiting for images...")
    rgb = armisaacsim_0.getImage("camera_rgb_0")
    depth = armisaacsim_0.getImage("camera_depth_0")
    time.sleep(0.2)

# Log images for debugging or record keeping
log("grid/rgb_img", rgb)
log("grid/depth_img", depth)

data = rgb.data

Waiting for images...


In [ ]:
# -----------------------------
# Run Object Detection
# -----------------------------
boxes, scores, labels = det_model.run(rgbimage=data[:, :, :3], prompt=obj)
while boxes is None or len(boxes) == 0:
    boxes, scores, labels = det_model.run(rgbimage=data[:, :, :3], prompt=obj)

# Select the bounding box with the highest confidence
i = np.argmax(scores)
center_x, center_y = get_box_center(boxes[i], [data.shape[1], data.shape[0]])
if center_x == -1 and center_y == -1:
    print("Object too big")

# -----------------------------
# Convert 2D Image Coordinates to 3D World Coordinates
# -----------------------------
# Extract depth at the center of the detected box
depth_data = depth.data[:, :, 0]
Z = depth_data[int(center_y), int(center_x)]
X_c = ((center_x - cx) / fx) * Z
Y_c = ((center_y - cy) / fy) * Z
Z_c = Z

# Create a homogeneous coordinate in the camera frame
camera_point_h = np.array([X_c, Y_c, Z_c, 1.0])
# Transform the point from the camera frame to the world frame
world_point_h = T_world_from_cam @ camera_point_h
X_w, Y_w, Z_w = world_point_h[:3]

# Apply an offset for proper grasping (e.g., half the cube's side)
goal_pos_np = np.array([X_w - 0.25, Y_w - 0.1, Z_w])

In [ ]:
# -----------------------------
# Move the Robot Arm in the XY Plane
# -----------------------------
for _ in range(50):
    curr_pos = armisaacsim_0.getPosition()
    # Wait until a valid current position is obtained
    while curr_pos is None or curr_pos.x_val is None or curr_pos.y_val is None:
        curr_pos = armisaacsim_0.getPosition()
        time.sleep(0.1)
    
    curr_pos_np = np.array([curr_pos.x_val, curr_pos.y_val, curr_pos.z_val])
    # Calculate the direction vector and step delta towards the goal
    direction = goal_pos_np - curr_pos_np
    delta_pos = direction * step_size

    # Command the arm to move in the XY direction
    armisaacsim_0.moveToDeltaPose(
        Position(delta_pos[0], delta_pos[1], 0.0),
        Orientation(*goal_quat_np)
    )
    time.sleep(0.1)

# -----------------------------
# Adjust the Arm's Z Position
# -----------------------------
for _ in range(50):
    curr_pos = armisaacsim_0.getPosition()
    while curr_pos is None or curr_pos.x_val is None or curr_pos.y_val is None:
        curr_pos = armisaacsim_0.getPosition()
        time.sleep(0.1)

    # Calculate the required Z-axis movement to reach goal_z
    delta_pos_z = (goal_z - curr_pos.z_val) * step_size_z
    armisaacsim_0.moveToDeltaPose(
        Position(0.0, 0.0, delta_pos_z),
        Orientation(*goal_quat_np)
    )
    time.sleep(0.1)